In [ ]:
import mediapipe as mp
import cv2
import numpy as np
from joblib import load

In [ ]:
model= load('randomforest.model')

In [ ]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

def calculate_angle(a, b, c):
    a = np.array(a)  
    b = np.array(b)  
    c = np.array(c)  
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360.0 - angle
        
    return angle

def extract_angles_from_video(video_path, angle_pairs):
    cap = cv2.VideoCapture(video_path)
    all_angles = []
    frame_num = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            frame_angles = []
            
            for (a, b, c) in angle_pairs:
                angle = calculate_angle(
                    [landmarks[a].x, landmarks[a].y],
                    [landmarks[b].x, landmarks[b].y],
                    [landmarks[c].x, landmarks[c].y]
                )
                frame_angles.append(angle)
            
            all_angles.append([frame_num] + frame_angles)
        
        frame_num += 1
    
    cap.release()
    return all_angles

angle_pairs = [
    (mp_pose.PoseLandmark.LEFT_SHOULDER.value, mp_pose.PoseLandmark.LEFT_ELBOW.value, mp_pose.PoseLandmark.LEFT_WRIST.value),
    (mp_pose.PoseLandmark.RIGHT_SHOULDER.value, mp_pose.PoseLandmark.RIGHT_ELBOW.value, mp_pose.PoseLandmark.RIGHT_WRIST.value),
    (mp_pose.PoseLandmark.LEFT_HIP.value, mp_pose.PoseLandmark.LEFT_KNEE.value, mp_pose.PoseLandmark.LEFT_ANKLE.value),
    (mp_pose.PoseLandmark.RIGHT_HIP.value, mp_pose.PoseLandmark.RIGHT_KNEE.value, mp_pose.PoseLandmark.RIGHT_ANKLE.value),
    (mp_pose.PoseLandmark.LEFT_SHOULDER.value, mp_pose.PoseLandmark.LEFT_HIP.value, mp_pose.PoseLandmark.LEFT_KNEE.value),
    (mp_pose.PoseLandmark.RIGHT_SHOULDER.value, mp_pose.PoseLandmark.RIGHT_HIP.value, mp_pose.PoseLandmark.RIGHT_KNEE.value),
    (mp_pose.PoseLandmark.LEFT_ELBOW.value, mp_pose.PoseLandmark.LEFT_SHOULDER.value, mp_pose.PoseLandmark.LEFT_HIP.value),
    (mp_pose.PoseLandmark.RIGHT_ELBOW.value, mp_pose.PoseLandmark.RIGHT_SHOULDER.value, mp_pose.PoseLandmark.RIGHT_HIP.value),
]

In [ ]:
def predict_exercises_in_video(video_path, angle_pairs, model, segment_length=30):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    input_file = video_path.split("/")[-1]
    output = f"{input_file}_result.mp4"
    out = cv2.VideoWriter(output, cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))  
    
    segment_angles = []
    frame_num = 0
    prediction = ''
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            frame_angles = []
            
            for (a, b, c) in angle_pairs:
                angle = calculate_angle(
                    [landmarks[a].x, landmarks[a].y],
                    [landmarks[b].x, landmarks[b].y],
                    [landmarks[c].x, landmarks[c].y]
                )
                frame_angles.append(angle)
            
            segment_angles.append(frame_angles)
        
        if len(segment_angles) == segment_length:
            segment_angles_np = np.array(segment_angles).mean(axis=0).reshape(1, -1)
            prediction = model.predict(segment_angles_np)[0]
            segment_angles = []
        
        cv2.putText(frame, f'Predicted Exercise: {prediction}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        out.write(frame)
        
        frame_num += 1
    
    cap.release()
    out.release()
    print(f'Output video saved as {output}')

video_path = 'C:/Users/jayak/Downloads/Python workspace/infosys final work/squat_29.mp4'
#location of input video
predict_exercises_in_video(video_path, angle_pairs, model, segment_length=30)